# This notebook contains examples to reproject MOLA MEGDR files to a new coordinate system. The primary motivation was to import MOLA elevation maps as rasters into SeisWare, however the method can transfer to other datasets

### These examples use MEG128 data downloaded from the PDS (https://pds-geosciences.wustl.edu/missions/mgs/megdr.html). The destination CRS was defined within SeisWare and exported as a .wkt file. <br><br>Example 1: Reproject a single MOLA MEGDR file  <br>Example2: Merge two MOLA MEGDR files that are vertically stacked (cover the same longitude range) and reproject the merged file  <br>Example 3: Define a cropped area of the merged MEGDR and write the cropped subset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

import rasterio
from rasterio import Affine
from rasterio.plot import show
import rasterio.merge
import rasterio.mask
import rasterio.crs
from rasterio.warp import calculate_default_transform, reproject, Resampling

from shapely.geometry import Polygon

## Tutorial date: October 2021
## Versions used:
##     Python: 3.9.7
##     Numpy: 1.21.2
##     Matplotlib: 3.4.3
##     Rasterio: 1.2.8
##     Shapely: 1.7.1

In [ ]:
##----------------------------------------
## EXAMPLE 1 - REPROJECT A SINGLE MOLA MEGDR
##
## This example will make a RGB image of MOLA
## elevation and write a reprojected .tif file
##----------------------------------------


##-----------------------------##
##-------- USER INPUTS --------##
##-----------------------------##

## Specify filepath to the exported .wkt file
wkt_filepath = './Phlegra_Test_UTM58N.wkt'


## Specify filepath to the MEGDR .lbl file from the PDS
## NOTE: you must have both the .lbl and .img file in
## the same directory, and `rasterio` expects you to 
## point at the .lbl file
megdr_filepath = './megt44n090hb.lbl'


## Specify the matplotlib colormap to use on the raster
## as well as the min/max values to assign to the colormap
colormap = 'terrain'
cmap_min = -6000
cmap_max = 4000


## Specify the output filename of the reprojected .tif
output_filepath = './reproject_test_UTM58_MOLA44.tif'



##-----------------------------##
##-----------------------------##
##-----------------------------##

## Create a CRS object from the .wkt file
with open(wkt_filepath, 'r') as crs_file:
    crs_data = crs_file.readlines()
    new_crs = crs_data[0].rstrip()
dst_crs = rasterio.crs.CRS.from_string(new_crs)


## Create a temporary .tif file of MOLA data
with rasterio.open(megdr_filepath) as src:
    tmp_raster = src.read(1)

    ## Save the raster data as a temporary .png file
    ## then define the RGB channels
    plt.imsave('./tmp_img_delete.png', tmp_raster, cmap=colormap, vmin=cmap_min, vmax=cmap_max)

    rgb_img = rasterio.open('./tmp_img_delete.png')
    rgb_img = rgb_img.read([1,2,3])
    rgb_img = rgb_img.astype('uint8')
    
    r = rgb_img[0,:,:]
    g = rgb_img[1,:,:]
    b = rgb_img[2,:,:]

    ## Create a temporary 3-channel .tif file
    with rasterio.open('./tmp_mola_delete.tif', 
                   'w', driver='GTiff', 
                   width=src.width, height=src.height, 
                   count=3,
                   dtype=r.dtype,
                   crs=src.crs,
                    transform=src.transform) as dst:
        for k, arr in [(1, r), (2, g), (3, b)]:
            dst.write(arr, indexes=k)
            

## Reproject the temporary .tif file into the
## new coordinate system
with rasterio.open('./tmp_mola_delete.tif') as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(output_filepath, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

## Remove the temporary files
os.remove('./tmp_img_delete.png')
os.remove('./tmp_mola_delete.tif')

In [ ]:
##----------------------------------------
## EXAMPLE 2 - MERGE TWO MOLA MEGDR MAPS
## AND REPROJECT TO A NEW COORDINATE SYSTEM
##
## This example will make a grayscale image of 
## MOLA elevation and write a reprojected .tif 
## file
##----------------------------------------


##-----------------------------##
##-------- USER INPUTS --------##
##-----------------------------##

## Specify filepath to the exported .wkt file
wkt_filepath = './Phlegra_Test_UTM58N.wkt'


## Specify filepaths to the MEGDR .lbl files from the PDS
## NOTE: you must have both the .lbl and .img file in
## the same directory, and `rasterio` expects you to 
## point at the .lbl file
megdr_filepath1 = './megt44n090hb.lbl'
megdr_filepath2 = './megt88n090hb.lbl'


## Specify the matplotlib colormap to use on the raster
## as well as the min/max values to assign to the colormap
colormap = 'gray'
cmap_min = -6000
cmap_max = 4000


## Specify the output filename of the reprojected .tif
output_filepath = './reproject_test_UTM58_Merged.tif'



##-----------------------------##
##-----------------------------##
##-----------------------------##


## Create a CRS object from the .wkt file
with open(wkt_filepath, 'r') as crs_file:
    crs_data = crs_file.readlines()
    new_crs = crs_data[0].rstrip()
dst_crs = rasterio.crs.CRS.from_string(new_crs)


## Create a temporary .tif file of MOLA data
with rasterio.open(megdr_filepath1) as src1:
    with rasterio.open(megdr_filepath2) as src2:
        tmp_raster1 = src1.read(1)
        tmp_raster2 = src2.read(1)
        
        ## A few checks on the example files, these may
        ## change depending on the exact use case
        assert src1.width == src2.width, "Width Mismatch"
        assert src1.crs == src2.crs, "CRS Mismatch"
        
        ## Merge the two rasters and create a properly-sized
        ## data object to write into a temporary raster
        tmp_merged = rasterio.merge.merge([src1, src2])
        write_data = np.squeeze(tmp_merged[0])

        ## Create a temporary raster of the merged data
        with rasterio.open('./tmp_MERGED_delete.tif', 
                       'w', driver='GTiff', 
                       width=src1.width, height=(src1.height + src2.height), 
                       count=1,
                       dtype=tmp_raster1.dtype,
                       crs=src1.crs,
                        transform=tmp_merged[1]) as dst:
            dst.write(write_data, 1)
            

## Open the merged raster and create a new temporary raster
## that is only one band in order to make a black and white 
## heightmap
with rasterio.open('./tmp_MERGED_delete.tif') as src:
    tmp_raster = src.read(1)

    # Make grayscale image of the merged data
    plt.imsave('./tmp_img_delete.png', tmp_raster, cmap='gray', vmin=cmap_min, vmax=cmap_max)

    rgb_img = rasterio.open('./tmp_img_delete.png')
    rgb_img = rgb_img.read([1,2,3])
    rgb_img = rgb_img.astype('uint8')

    ## Since we saved the image with a 'gray' colormap, 
    ## we can grab any single channel to write the raster
    rgb_img = rgb_img[0,:,:]

    with rasterio.open('./tmp_MERGED_delete2.tif', 
                   'w', driver='GTiff', 
                   width=src.width, height=src.height, 
                   count=1,
                   dtype=tmp_raster.dtype,
                   crs=src.crs,
                    transform=src.transform) as dst:
        dst.write(rgb_img, 1)
        
with rasterio.open('./tmp_MERGED_delete2.tif') as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(output_filepath, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
            
## Remove the temporary files
os.remove('./tmp_img_delete.png')
os.remove('./tmp_MERGED_delete.tif')
os.remove('./tmp_MERGED_delete2.tif')

In [ ]:
##----------------------------------------
## EXAMPLE 3 - CROP THE MERGED MEGDR
##
## This example will use a polygon to define XY
## coordinates of our cropped area on the reprojected
## merged MEGDR created in Example 2
##----------------------------------------


##-----------------------------##
##-------- USER INPUTS --------##
##-----------------------------##


## Specify filepath to the merged MEGDR file
megdr_filepath = './reproject_test_UTM58_Merged.tif'

## Define the corner points of the cropping polygon
subset_polygon = Polygon([(0, 1.5e6), (0, 3.1e6), (1e6, 3.1e6), (1e6, 1.5e6)])

## Specify the output filename of the reprojected .tif
output_filepath = './reproject_test_UTM58_Cropped.tif'


##-----------------------------##
##-----------------------------##
##-----------------------------##

## Use the defined polygon to mask the merged .tif
with rasterio.open(megdr_filepath) as src:
    out_image, out_transform = rasterio.mask.mask(src, [subset_polygon], crop=True)
    out_meta = src.meta
    
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

## Show the polygon on top of the merged MEGDR to confirm
## the cropping coordinates are correct
with rasterio.open(megdr_filepath) as src:
    
    fig, axM = plt.subplots(1,1,figsize=(10,10))
    show(src, ax=axM, cmap='gray')
    axM.grid('both')
    plt.plot(*subset_polygon.exterior.xy)
    plt.show()
    
    
## Write the new cropped .tif file
with rasterio.open(output_filepath, "w", **out_meta) as dest:
    dest.write(out_image)